In [59]:
import threading
from datetime import datetime
from datetime import timedelta
from datetime import timezone as tzone
import time


In [64]:
class Scheduler:
    """
    A class to represent a schedule for jobs with start and end dates.
    
    Attributes:
    -----------
    threading : boolean 
        Whether or not to use threading while running the jobs 
    
    Methods:
    --------
    job():
        Method to create and configure a job
    run_all():
        Runs all jobs with or without threading
    """

    def __init__(self, threading = False, startdate = None, timezone=None):
        
        if not isinstance(timezone, int) and timezone is not None:
            raise ValueError("Timezone must be an integer")
        
        if not isinstance(startdate, str) and startdate is not None:
            raise ValueError("Date must be an str format %Y-%m-%d %H:%M")
       
        self.threads = threading
        self.timezonesc = tzone(timedelta(hours=timezone)) if timezone is not None else datetime.now().astimezone().tzinfo
        self.startdatesc = (datetime.strptime(startdate, '%Y-%m-%d %H:%M') if startdate else datetime.now().replace(second=0, microsecond=0)).astimezone(self.timezonesc)
        

        # validating that passed startdate isn't more than current time
        if  self.startdatesc.strftime('%Y-%m-%d %H:%M') < datetime.now().strftime('%Y-%m-%d %H:%M'):
            raise ValueError("Startdate cannot be less than the current date")
        
        self.jobs = []  #idk if i am gonna use it at all 



    def job(self, startdate=None, timezone=None):
        
        if not isinstance(timezone, int) and timezone is not None:
            raise ValueError("Timezone must be an integer")
        
        if not isinstance(startdate, str) and startdate is not None:
            raise ValueError("Date must be an str fromat %Y-%m-%d %H:%M")

        timezonej = tzone(timedelta(hours=timezone)) if timezone is not None else  self.timezonesc
        startdatej = datetime.strptime(startdate, '%Y-%m-%d %H:%M').astimezone(self.timezonesc) if startdate else self.startdatesc
            
            
        if  startdatej.strftime('%Y-%m-%d %H:%M') < datetime.now().strftime('%Y-%m-%d %H:%M'):
            raise ValueError("Job start date cannot be less than the current date")


        job = {
            'startdate': startdatej,
            'time_zone': timezonej,
            'enddate': None,
            'func': None,
            'name': None,
            'unit': None,
            'interval': None,
            'next_run': None,
            'repeats': None,  # New attribute for repeats
            'repeat_count': 0,  # Counter for completed runs
            'args': (),
            'kwargs': {}
        }
        
        
        class JobWrapper:
            def __init__(self, job):
                self.job = job
            @property
            def second(self):
                self.job['unit'] = 'second'
                # print('in second')
                # print(f'dis is stdate {jstartdate}')
                return self

            @property
            def minute(self):
                self.job['unit'] = 'minute'
                return self
            
            @property
            def hour(self):
                self.job['unit'] = 'hour'
                return self
            
            # @property
            def day(self, at=None):
                print('inday')
                self.job['unit'] = 'day'
                self.job['at'] = at  # Store the `at` attribute for use in the next run calculation
                if at:
                    # Parse the time in HH:MM format and update the next run time accordingly
                    hour, minute = map(int, at.split(':'))
                    today = datetime.now(self.job['time_zone']).replace(hour=hour, minute=minute, second=0, microsecond=0)
                    # If the specified time has already passed today, set it for the next day
                    if today <= datetime.now(self.job['time_zone']):
                        today += timedelta(days=1)
                    self.job['next_run'] = today
                return self
            
            
            def do(self, func, name):
                self.job['func'] = func
                self.job['name'] = name
                # print('indo')
                return self
            
            def every(self, interval):
                self.job['interval'] = interval
                # print('inevery')
                return self

            def until(self, enddate):
                enddate = datetime.strptime(enddate, '%Y-%m-%d %H:%M')
                enddate = enddate.astimezone(self.job['time_zone'])
                if enddate < self.job['startdate']:
                    raise ValueError('Startdate must be earlier than Enddate')
                self.job['enddate'] = enddate
                print('inuntil')
                return self

            def repeat(self, times):
                """Set the number of times the job should repeat."""
                self.job['repeats'] = times
                return self

            def calculate_next_run(self, current_time):
                # Truncate the current_time to remove milliseconds
                current_time = current_time.replace(microsecond=0)
            
                if self.job['unit'] == 'second':
                    self.job['next_run'] = current_time + timedelta(seconds=self.job['interval'])
                elif self.job['unit'] == 'minute':
                    current_time = current_time.replace(second=0)
                    self.job['next_run'] = current_time + timedelta(minutes=self.job['interval'])
                elif self.job['unit'] == 'hour':
                    current_time = current_time.replace(minute=0, second=0)
                    self.job['next_run'] = current_time + timedelta(hours=self.job['interval'])
                elif self.job['unit'] == 'day':
                    # Handling the 'at' parameter for the day interval
                    if 'at' in self.job and self.job['at']:
                        # Parse the 'at' time from the job settings
                        hour, minute = map(int, self.job['at'].split(':'))
                        next_run = current_time.replace(hour=hour, minute=minute, second=0, microsecond=0)
            
                        # Ensure next_run is set on the correct day interval
                        if next_run <= current_time:
                            next_run += timedelta(days=self.job['interval'])
                        self.job['next_run'] = next_run
                    else:
                        # Default behavior without 'at' attribute
                        current_time = current_time.replace(hour=0, minute=0, second=0)
                        self.job['next_run'] = current_time + timedelta(days=self.job['interval'])
                else:
                    raise ValueError("Unsupported unit. Please extend the `calculate_next_run` method to support other units.")
                print(f"calc next run: {self.job['next_run']}")
            
                return self.job['next_run']

            def run(self, *args, **kwargs):
                # Wait until jstartdate occurs
                now = datetime.now(self.job['time_zone']).replace(microsecond=0)
                if now < self.job['startdate']:
                    wait_time = (self.job['startdate'] - now).total_seconds()
                    print(f'Waiting for {wait_time} seconds to')
                    time.sleep(wait_time)
        
                # Check if enddate is today and at time is in the past
                if self.job['enddate'] and self.job['enddate'].date() == now.date() and 'at' in self.job and self.job['at']:
                    hour, minute = map(int, self.job['at'].split(':'))
                    at_time = now.replace(hour=hour, minute=minute, second=0, microsecond=0)
                    if at_time < now:
                        raise ValueError("The specified 'at' time is earlier than the current time for today's enddate.")
        
                while True:
                    now = datetime.now(self.job['time_zone']).replace(microsecond=0)
                    if self.job['next_run'] is None or now >= self.job['next_run']:
                        self.calculate_next_run(now)
                        print(f'next run is none or now something ')
                        if self.job['next_run'].strftime('%Y-%m-%d %H:%M') == self.job['enddate'].strftime('%Y-%m-%d %H:%M'):
                            print('in first break')
                            break
                        if self.job['func']:
                            print(f'in fucntion execution')
                            self.job['func'](*self.job['args'], **self.job['kwargs'])
                            self.job['repeat_count'] += 1
                            print(f'this is repeat count: {self.job["repeat_count"]}, next run: {self.job["next_run"]}')
                            if self.job['repeats'] is not None and self.job['repeat_count'] >= self.job['repeats']:
                                print('repeatbreak?')
                                break
                    if self.job['enddate'] and now >= self.job['enddate']:
                        print(f"endd{self.job['enddate']} , now {now}")
                        print('in second break')
                        break
                    time.sleep(1)

                    # time.sleep(1)
                    
                    
                    
        job_wrapper = JobWrapper(job)
        self.jobs.append(job_wrapper)
        return job_wrapper
    
    def run_all(self):
        if self.threads:
            threads = []
            for job in self.jobs:
                thread = threading.Thread(target=job.run)
                threads.append(thread)
                thread.start()
            for thread in threads:
                thread.join()
        else:
            for job in self.jobs:
                job.run()



In [65]:


# Example functions
def report1(num1=1,num2 = 2):
    print(num1+num2)

# def report2():
#     print("Function report2 is running.")
# 
# def report3():
#     print("Function report3 is running.")

In [66]:
scheduler = Scheduler(threading=False, timezone=5, startdate="2024-08-31 22:50")

scheduler.job(timezone=4).do(report1, 'report1').every(1).second().repeat(5).until('2024-08-31 21:50')


# scheduler.job(startdate='2024-08-22 17:10')
# scheduler.job()
# scheduler.job(startdate='2024-08-22 20:10', timezone=7)

scheduler.run_all()


TypeError: 'JobWrapper' object is not callable

In [58]:
"""
job დიქშინარი და ფუნქცია ერთი და იგივე სახელითაა და ჯობია განსხვავდებოდეს

jstartdate, scstartdate, sctimezone და jtimezone-ს უეჭველი სხვა რამე უნდა დავარქვათ (სასურველია თუ ერთნაირი იქნება ორივესთვის) ალბათ პირდაპირ იმიტო არ დაარქვი რო იმპორტში შემოგყავს მაგრამ ცადე იმპორტიდან as-ით შეიყვანო

"""

'\njob დიქშინარი და ფუნქცია ერთი და იგივე სახელითაა და ჯობია განსხვავდებოდეს\n\njstartdate, scstartdate, sctimezone და jtimezone-ს უეჭველი სხვა რამე უნდა დავარქვათ (სასურველია თუ ერთნაირი იქნება ორივესთვის) ალბათ პირდაპირ იმიტო არ დაარქვი რო იმპორტში შემოგყავს მაგრამ ცადე იმპორტიდან as-ით შეიყვანო\n\n'

In [ ]:
"""

self.threads = threading  
self.startdatesc = datetime.strptime(scstartdate, '%Y-%m-%d %H:%M')  if scstartdate else datetime.strptime(datetime.now().strftime('%Y-%m-%d %H:%M'), '%Y-%m-%d %H:%M') 

if sctimezone is not None:
    self.timezonesc = timezone(timedelta(hours=sctimezone))
    self.startdatesc = self.startdatesc.astimezone(self.timezonesc)
else:
    self.timezonesc = datetime.now().astimezone().tzinfo
    self.startdatesc = self.startdatesc.astimezone(self.timezonesc)


ალბათ უკეთესი იქნება ამის მაგივრად ვიხმაროთ შემდეგი ლოგიკა, რათა თავი ავარიდოთ 



# threading-ი სტანდარტულად
self.threads = threading

# ჯერ გავიგოთ თუ რომელ თაიმზონასთან გვაქ საქმე
self.timezonesc = timezone(timedelta(hours=sctimezone)) if sctimezone is not None else datetime.now().astimezone().tzinfo

# იმის შემდეგ რაც გვცოდინება თაიმზონა პირდაპირ განვმარტოთ დაწყების თარითი
self.startdatesc = (datetime.strptime(scstartdate, '%Y-%m-%d %H:%M') if scstartdate else datetime.now().replace(second=0, microsecond=0)).astimezone(self.timezonesc)

                                        
# ან თუ ფიქრობ რომ ერთი ხაზისთვის ძალიან დიდი კოდია შეგვიძლია გავყოთ 2 ნაწილად
start_date_str = scstartdate if scstartdate else datetime.now().strftime('%Y-%m-%d %H:%M')
self.startdatesc = datetime.strptime(start_date_str, '%Y-%m-%d %H:%M').astimezone(self.timezonesc)


"""

In [ ]:
"""


"""

In [ ]:
"""
job დიქშინარი და ფუნქცია ერთი და იგივე სახელითაა და ჯობია განსხვავდებოდეს

jstartdate, scstartdate, sctimezone და jtimezone-ს უეჭველი სხვა რამე უნდა დავარქვათ (სასურველია თუ ერთნაირი იქნება ორივესთვის) ალბათ პირდაპირ იმიტო არ დაარქვი რო იმპორტში შემოგყავს მაგრამ ცადე იმპორტიდან as-ით შეიყვანო

"""